# A. Imports and initial setup

## A.1 import data

In [2]:
import numpy as np 
import pandas as pd 

from utils import get_stopwords, get_stemmer, tokenizer, data2idx, data2bow, matrix2txt, bow2dist
dataset_location='../../locality-sensitive-hashing/datasets/development.json'
# dataset_location='../CorpusTrabalho4.zip'
df = pd.read_json(dataset_location, orient='records')
# print(df.count()) 9998
df.sample(10).head(10)

,description,id,title
820,Dados da Vaga de EmpregoVendedorRequisitos/Per...,820,Emprego vendedor de produtos pets e agropecuarios
6317,Gerente de loja\nAtividades:\nSupervisionar as...,6317,Vaga para Gerente de Loja
4417,:jotapecontabil.com.brjotape contabilidade est...,4417,Vaga para Assistente de Auditor
6274,nome fantasia:tryphotels.com.br\nhotel tryp es...,6274,Vaga para Auxiliar de Cozinha
5428,Necessario que tenha CNH B e ensino medio comp...,5428,AUXILIAR DE ESCRIToRIO
5695,"Distribuidora kretzer, empresa solida no merca...",5695,Vaga para Representante Comercial
8562,Requisitos:\n\n\n\n- Experiencia na funcao.\n\...,8562,PEDREIRO
2325,Vaga de tecnico de cabeamento estruturado com ...,2325,Vaga para Agente de Suporte Tecnico
3829,Mecanico Realizara manutencao e reparos em vei...,3829,Mecanico
1662,Eletricista montador com experiencia em montag...,1662,Vaga para Eletricista Industrial


## A.2 Stopwords

In [3]:
stopwords= list(get_stopwords())
ind = np.random.randint(0,len(stopwords), size=5)
print('total stop words are %d' % (len(stopwords)))
print([stopwords[i] for i in ind])      

total stop words are 203
['delas', 'houvera', 'houvemos', 'pelos', 'estivermos']


## A.3 Stemmer

In [4]:
stemmer= get_stemmer() 
words_test= ['Programador', 'Programa', 'pegaram', 'correram']
print('Before stemming:')
print(words_test)
print('After stemming:')
print([stemmer.stem(w) for w in words_test])

Before stemming:
['Programador', 'Programa', 'pegaram', 'correram']
After stemming:
['program', 'program', 'peg', 'corr']


## A.4 Tokenize the title

In [5]:
tokenfy = lambda x : tokenizer(x, stopwords= stopwords, stemmer=stemmer)
df['token_title'] = df['title'].transform(tokenfy)
df.sample(10).head(10)

,description,id,title,token_title
9611,Precisamos de professores para trabalhar de 1ª...,9611,Vaga para Professor,vag professor
6204,Sobre a vaga \r\n\r\n\r\n Descricao\r\n\r\n...,6204,Vaga de Desenvolvedora Java,vag desenvolvedor jav
6028,Porteiro\nLocal de trabalho ? centro ? rj\nFor...,6028,Vaga para Porteiro,vag porteir
6171,Pre-requisitos:\nEnsino medio completo;\nNeces...,6171,Vaga para Vendedor,vag vendedor
2553,***Zelador/auxiliar de manutencao ***Empresa: ...,2553,Vaga para Auxiliar de Manutencao,vag auxili manutenca
8109,Requisitos:\n de no minimo 1 na funcao / corel...,8109,Vaga para Arte Finalista,vag arte final
3613,Ira receber informacoes e autorizacoes de fatu...,3613,Assistente financeiro,assistent financeir
7409,Sobre a vaga \r\n\r\n\r\n Descricao\r\n\r\n...,7409,Vaga de Editor De Imagem Jr,vag editor imag jr
9604,Atividades pertinentes ao cargo.\n\nEscolarida...,9604,Coordenador(a) de Vendas,coorden vend
7153,Tecnico Mecanica \n\nResponsabilidades: \n- De...,7153,Tecnico Mecanica,tecnic mecan


## A.5 Tokenize the description

In [8]:
df['token_description'] = df['description'].transform(tokenfy)
# df.sample(10).head(10)
df.head(10)

,description,id,title,token_title,token_description
0,Auxilia nas rotinas do departamento administra...,0,Aprendiz Administrativo,aprendiz administr,auxil rotin depart administr acompanh relatori...
1,\r\n Sera responsavel pelo dese...,1,Programador(a),program,ser responsavel desenvolv manutenca sit loj v...
2,Sera responsavel pelo desenvolvimento e manute...,2,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
3,Realizar conciliacao de contas contabeis. Anal...,3,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
4,Realizar conciliacao de contas contabeis. Anal...,4,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
5,Sera responsavel pelo desenvolvimento e manute...,5,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
6,Realizar conciliacao de contas contabeis. Anal...,6,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
7,Realizar conciliacao de contas contabeis. Anal...,7,Estagiario(a) Contabilidade,estagiario contabil,realiz conciliaca cont contab analis demonstra...
8,Sera responsavel pelo desenvolvimento e manute...,8,Programador(a),program,ser responsavel desenvolv manutenca sit loj vi...
9,A Empresa Tok&Stok conta com mais de 4.000 fun...,9,Vendedor Cadista - Copacabana,vendedor cadist copacaban,empres tokstok cont 4000 funcionari distribu 5...


## A.6 Token 2 idx

In [9]:
word2idx={}
df = data2idx(df, word2idx)
# df.sample(10).head(10)
df.head(10)

document:9997 of 9998	VOCAB:25671	WORD COUNT:574479	


,description,id,title,token_title,idx_description
0,Auxilia nas rotinas do departamento administra...,0,Aprendiz Administrativo,aprendiz administr,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17
1,\r\n Sera responsavel pelo dese...,1,Programador(a),program,18 19 20 21 22 23 24 25 26 27 28 29 21 30 22 3...
2,Sera responsavel pelo desenvolvimento e manute...,2,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
3,Realizar conciliacao de contas contabeis. Anal...,3,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
4,Realizar conciliacao de contas contabeis. Anal...,4,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
5,Sera responsavel pelo desenvolvimento e manute...,5,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
6,Realizar conciliacao de contas contabeis. Anal...,6,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
7,Realizar conciliacao de contas contabeis. Anal...,7,Estagiario(a) Contabilidade,estagiario contabil,40 41 42 43 44 45 46 47 7 48 49 50 46
8,Sera responsavel pelo desenvolvimento e manute...,8,Programador(a),program,19 20 21 22 23 24 25 26 27 28 29 21 30 22 31 3...
9,A Empresa Tok&Stok conta com mais de 4.000 fun...,9,Vendedor Cadista - Copacabana,vendedor cadist copacaban,51 52 42 53 54 55 56 24 57 58 59 60 61 62 63 6...


In [10]:
bow = data2bow(df, word2idx)
print(bow.shape)
bow[:5,:5]

(25671, 9998)


array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]], dtype=int32)

In [11]:
bow[:10,:10]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 1, 1, 0, 1, 1, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]], dtype=int32)

In [ ]:
dist=bow2dist(bow)

In [26]:
dist[:10,:10]

array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 7.        ,  0.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 6.92820311,  1.        ,  0.        ,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 5.56776428,  6.78233004,  6.70820379,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 5.56776428,  6.78233004,  6.70820379,  0.        ,  0.        ,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 6.92820311,  1.        ,  0.        ,  6.70820379,  6.70820379,
         0.        ,  0.        ,  0.        ,  0.        ,  0.        ],
       [ 5.56776428,  6.78233004,  6.70820379,  0.        ,  0.        ,
         6.70820379,  0.        ,  0.        

In [ ]:
matrix2txt(dist, filename='distance_matrix.txt')